# Init

In [1]:
%matplotlib inline

import csv
import itertools
import math
import matplotlib
import time
import logging
import sys
import os
import random
import warnings

import gensim
from gensim.models import KeyedVectors

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict

from sklearn import random_projection
from sklearn.manifold import TSNE
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, spdiags
from scipy.io import loadmat, savemat
from scipy.spatial.distance import cosine
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize, MultiLabelBinarizer

In [2]:
from fastrp_exp import *

In [3]:
from sklearn import preprocessing

In [4]:
from scoring import scoring

In [5]:
# to ignore sklearn warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [6]:
%load_ext autoreload
%autoreload 2

# Load Data

## WWW-10K

In [7]:
www = loadmat('example_graphs/www-10k.mat')
A = www['network']
N = A.shape[0]
A

<10000x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 7809220 stored elements in Compressed Sparse Column format>

In [8]:
G = nx.from_scipy_sparse_matrix(A)

In [9]:
int_nodes = list(map( int, G.nodes()) )

In [10]:
min(int_nodes), max(int_nodes), len(int_nodes)

(0, 9999, 10000)

In [11]:
G.number_of_nodes(), G.number_of_edges()

(10000, 3904610)

# Sample a Subgraph

In [12]:
import networkx as nx
from scipy.sparse import find
from random import sample
from scipy.spatial.distance import pdist, squareform

In [13]:
tree = nx.minimum_spanning_tree(G)
G_nodes, G_edges, tree_edges = G.nodes(), G.edges(), tree.edges()
n_edges_70 = int(0.7*len(G_edges))

In [14]:
non_tree_edges = [edge for edge in G_edges if edge not in tree_edges]

In [15]:
sampled_edges = sample(non_tree_edges, n_edges_70 - len(tree.edges()))
tree.add_edges_from(sampled_edges, weight=1.0)

In [19]:
A_70 = nx.to_scipy_sparse_matrix(tree)
savemat('example_graphs/www-10k-70.mat', {'network': A_70})
nx.write_edgelist(tree.to_directed(), open('example_graphs/www-10k-70.edgelist', 'wb'), data=['weight'])

In [17]:
%time edges_to_test = [edge for edge in G.edges() if edge not in tree.edges()]

CPU times: user 7.22 s, sys: 172 ms, total: 7.39 s
Wall time: 7.39 s


In [18]:
#Artificially removed:
# edges_to_test = [edge for edge in G.edges() if edge not in tree.edges()]

#Edges that never existed
# negatives = []
# while len(negatives)<len(edges_to_test):
#     rand_edge = sample(G_nodes, 2)
#     if rand_edge not in G_edges:
#         negatives.append(rand_edge)

#Random sample from existing/non-existing edges
%time rand_pairs = [sample(G_nodes, 2) for i in range(len(edges_to_test))]

CPU times: user 1min 39s, sys: 464 ms, total: 1min 40s
Wall time: 1min 40s


# FastRP

In [31]:
import optuna

In [32]:
prefix = 'result/blog'

In [33]:
%%time

def objective(trial):
    order_range = 1
    # Invoke suggest methods of a Trial object to generate hyperparameters.
    # weights = [trial.suggest_loguniform('weight' + str(order), 1.0, 64.0) for order in range(order_range)]
    weights = [trial.suggest_loguniform('weight' + str(order), 0.001, 1) for order in range(order_range)]
    alpha = trial.suggest_uniform('alpha', -0.4, 0.4)
    conf = {
        'projection_method': 'sparse',
        'input_matrix': 'adj',
        'weights': [1.0] + weights,
        'normalization': False,
        'dim': 512,
        'alpha': alpha,
        'C': 0.1
    }
    print(conf)
    # emb_filename = get_emb_filename(prefix, conf)
    # print (emb_filename)

    U = fastrp_wrapper(A_70, conf)
    # distances_negative = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in negatives])
    distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
    distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])
    
    # scores_negative = (distances_pos<distances_negative).sum() / len(distances_negative)
    scores_random = (distances_pos<distances_random).sum() / len(distances_random)
    return -scores_random

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=20)  # Invoke optimization of the objective function.

{'projection_method': 'sparse', 'input_matrix': 'adj', 'weights': [1.0, 0.01020790116922033], 'normalization': False, 'dim': 512, 'alpha': 0.34518438541539986, 'C': 0.1}


[I 2019-05-21 22:52:46,685] Finished trial#0 resulted in value: -0.8266442316475482. Current best value is -0.8266442316475482 with parameters: {'weight0': 0.01020790116922033, 'alpha': 0.34518438541539986}.


{'projection_method': 'sparse', 'input_matrix': 'adj', 'weights': [1.0, 0.8312164896441353], 'normalization': False, 'dim': 512, 'alpha': -0.1846332832645933, 'C': 0.1}


[I 2019-05-21 22:54:16,104] Finished trial#1 resulted in value: -0.6882778732489715. Current best value is -0.8266442316475482 with parameters: {'weight0': 0.01020790116922033, 'alpha': 0.34518438541539986}.


{'projection_method': 'sparse', 'input_matrix': 'adj', 'weights': [1.0, 0.500369686400032], 'normalization': False, 'dim': 512, 'alpha': -0.291666913633987, 'C': 0.1}


...

[I 2019-05-21 23:13:05,578] Finished trial#15 resulted in value: -0.8437027001416275. Current best value is -0.8478687158683368 with parameters: {'weight0': 0.0015491215188953424, 'alpha': 0.16289814172057032}.


{'projection_method': 'sparse', 'input_matrix': 'adj', 'weights': [1.0, 0.003346007555272974], 'normalization': False, 'dim': 512, 'alpha': 0.38172264993900656, 'C': 0.1}


[I 2019-05-21 23:14:24,937] Finished trial#16 resulted in value: -0.8485047162200579. Current best value is -0.8485047162200579 with parameters: {'weight0': 0.003346007555272974, 'alpha': 0.38172264993900656}.


{'projection_method': 'sparse', 'input_matrix': 'adj', 'weights': [1.0, 0.0028655582323664575], 'normalization': False, 'dim': 512, 'alpha': 0.3798034734695112, 'C': 0.1}


KeyboardInterrupt: 

In [34]:
study.best_trial

FrozenTrial(number=16, state=<TrialState.COMPLETE: 1>, value=-0.8485047162200579, datetime_start=datetime.datetime(2019, 5, 21, 23, 13, 5, 582303), datetime_complete=datetime.datetime(2019, 5, 21, 23, 14, 24, 936457), params={'weight0': 0.003346007555272974, 'alpha': 0.38172264993900656}, user_attrs={}, system_attrs={'_number': 16}, intermediate_values={}, params_in_internal_repr={'weight0': 0.003346007555272974, 'alpha': 0.38172264993900656}, trial_id=16)

In [35]:
conf = {
        'projection_method': 'sparse',
        'input_matrix': 'adj',
        'weights': [1.0, 0.003346],
        'normalization': False,
        'dim': 512,
        'alpha': 0.382,
        'C': 0.1
}

In [37]:
%%time
U = fastrp_wrapper(A_70, conf)
distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])

scores_random = (distances_pos<distances_random).sum() / len(distances_random)
print (scores_random)

0.848502155144816
CPU times: user 1min 24s, sys: 1.09 s, total: 1min 25s
Wall time: 1min 21s


# RandNE

Run RandNE:

In [ ]:
python3 /home/hcchen/RandNE-Python/src/randne.py \
--input /home/hcchen/fast-random-projection/example_graphs/www-10k-70.mat \
--output /home/hcchen/fast-random-projection/result/www-10k-70-randne-emb.mat -q 2 -d 512 --weights 1 0.01

In [29]:
%%time
U = loadmat('/home/hcchen/fast-random-projection/result/www-10k-70-randne-emb.mat')['emb']
distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])

scores_random = (distances_pos<distances_random).sum() / len(distances_random)
print (scores_random)

0.8195124907908002
CPU times: user 1min 53s, sys: 340 ms, total: 1min 53s
Wall time: 1min 53s


# LINE

Compile the code:

In [ ]:
g++ -I /home/hcchen/gsl/include -L /home/hcchen/gsl/lib -lm -pthread -Ofast -march=native -Wall -funroll-loops -ffast-math -Wno-unused-result line.cpp -o line -lgsl -lm -lgslcblas
g++ -I /home/hcchen/gsl/include -L /home/hcchen/gsl/lib -lm -pthread -Ofast -march=native -Wall -funroll-loops -ffast-math -Wno-unused-result reconstruct.cpp -o reconstruct
g++ -I /home/hcchen/gsl/include -L /home/hcchen/gsl/lib -lm -pthread -Ofast -march=native -Wall -funroll-loops -ffast-math -Wno-unused-result normalize.cpp -o normalize
g++ -I /home/hcchen/gsl/include -L /home/hcchen/gsl/lib -lm -pthread -Ofast -march=native -Wall -funroll-loops -ffast-math -Wno-unused-result concatenate.cpp -o concatenate

First add the GSL library to path in shell:

In [ ]:
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/hcchen/gsl/lib

Run LINE:

In [ ]:
cd ~/LINE/linux && nohup ./line -train /home/hcchen/fast-random-projection/example_graphs/www-10k-70.edgelist \
-output /home/hcchen/fast-random-projection/result/www-10k-70.line.emb \
-binary 0 -size 200 -order 2 -negative 5 -samples 1000 -rho 0.025 -threads 40 &

Link prediction:

In [25]:
conf = {
        'projection_method': 'sparse',
        'input_matrix': 'adj',
        'weights': [1.0, 0.01],
        'normalization': False,
        'dim': 512,
        'alpha': 0.0,
        'C': 0.1
}

In [44]:
cosine(U[2], U[123])

0.9752370957285166

In [38]:
%%time
line_scores_negative, line_scores_random = [], []
model = KeyedVectors.load_word2vec_format('/home/hcchen/fast-random-projection/result/www-10k-70.line.emb')
U = np.asarray([model[str(node)] for node in range(len(model.vocab) )])
# print (U.shape)

distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])

scores_random = (distances_pos<distances_random).sum() / len(distances_random)
print (scores_random)

0.5124165196182632
CPU times: user 2min 56s, sys: 672 ms, total: 2min 57s
Wall time: 2min 59s


# DeepWalk

Run the following command in shell:

In [ ]:
nohup time deepwalk --format mat --input example_graphs/www-10k-70.mat \
--max-memory-data-size 0 --number-walks 80 --representation-size 128 --walk-length 40 --window-size 10 \
--workers 40 --output result/www-10k-70.deepwalk.emb &

Link prediction:

In [55]:
model = KeyedVectors.load_word2vec_format('/home/hcchen/fast-random-projection/result/www-10k-70.deepwalk.emb')
U = np.asarray([model[str(node)] for node in range(len(model.vocab) )])
# print (U.shape)

distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])

scores_random = (distances_pos<distances_random).sum() / len(distances_random)
print (scores_random)

0.786884392209892


# Node2vec

Run the following command in shell:

In [ ]:
cd /home/hcchen/deepwalk-sgns/deepwalk && nohup time python __main__.py --format mat \
--input /home/hcchen/fast-random-projection/example_graphs/www-10k-70.mat \
--max-memory-data-size 0 --number-walks 80 --representation-size 128 --walk-length 40 --window-size 10 \
--workers 40 --output /home/hcchen/deepwalk-sgns/example_graphs/www-10k-70.node2vec.emb &

Link prediction:

In [58]:
model = KeyedVectors.load_word2vec_format('/home/hcchen/deepwalk-sgns/example_graphs/www-10k-70.node2vec.emb')
U = np.asarray([model[str(node)] for node in range(len(model.vocab) )])
# print (U.shape)

distances_random = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in rand_pairs])
distances_pos = np.array([cosine(U[edge[0]], U[edge[1]]) for edge in edges_to_test])

scores_random = (distances_pos<distances_random).sum() / len(distances_random)
print (scores_random)

0.7504949277904835
